This notebook started the work of dealing with Mindat classifications. I need to revisit.

In [1]:
import os
import requests
import pandas as pd
import html
import re
from wbmaker import WikibaseConnection

In [2]:
def get_super(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    super_s = "ᴬᴮᶜᴰᴱᶠᴳᴴᴵᴶᴷᴸᴹᴺᴼᴾQᴿˢᵀᵁⱽᵂˣʸᶻᵃᵇᶜᵈᵉᶠᵍʰᶦʲᵏˡᵐⁿᵒᵖ۹ʳˢᵗᵘᵛʷˣʸᶻ⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾"
    res = x.maketrans(''.join(normal), ''.join(super_s))
    return x.translate(res)

def get_sub(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    sub_s = "ₐ₈CDₑբGₕᵢⱼₖₗₘₙₒₚQᵣₛₜᵤᵥwₓᵧZₐ♭꜀ᑯₑբ₉ₕᵢⱼₖₗₘₙₒₚ૧ᵣₛₜᵤᵥwₓᵧ₂₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎"
    res = x.maketrans(''.join(normal), ''.join(sub_s))
    return x.translate(res)

def wb_label(label_str):
    label = html.unescape(label_str)
    for sub in re.findall("<sub>[a-zA-z0-9]*</sub>", label):
        sub_chars = re.search('<sub>(.*)</sub>', sub)
        unicode_chars = "".join([get_sub(c) for c in sub_chars.group(1)])
        label = label.replace(sub, unicode_chars)
    for sup in re.findall("<sup>[a-zA-z0-9]*</sup>", label):
        sup_chars = re.search('<sup>(.*)</sup>', sup)
        unicode_chars = "".join([get_super(c) for c in sup_chars.group(1)])
        label = label.replace(sup, unicode_chars)

    label = label.replace("<middot>","·")
    label = label.replace("</sub>", "")
    return label

def pull_mindat_records(mindat_api):
    headers = {'Authorization': f"Token {os.environ['MINDAT_API_KEY']}"}
    all_records = []
    page_num = 1

    while page_num > 0:
        params = {
            'page_size': '100',
            'page': str(page_num),
            'format': 'json',
        }
        x = requests.get(mindat_api, params=params, headers=headers)
        if isinstance(x, list) and len(x) > 0:
            return x
        if "results" in x and x["results"]:
            all_records.extend(x["results"])
            if x["next"]:
                page_num += 1
            else:
                page_num = 0
        else:
            page_num = 0

    return all_records

In [3]:
strunz_classes = pull_mindat_records("https://api.mindat.org/nickel-strunz-10/classes/")
strunz_subclasses = pull_mindat_records("https://api.mindat.org/nickel-strunz-10/subclasses/")
strunz_families = pull_mindat_records("https://api.mindat.org/nickel-strunz-10/families/")

# dana_groups = pull_mindat_records("https://api.mindat.org/dana-8/groups/")
# dana_subgroups = pull_mindat_records("https://api.mindat.org/dana-8/subgroups/")

SSLError: HTTPSConnectionPool(host='api.mindat.org', port=443): Max retries exceeded with url: /nickel-strunz-10/classes/?page_size=100&page=1&format=json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))